In [ ]:
]activate ../..

In [ ]:
using Gen
using GenWorldModels

In [ ]:
include("../main.jl")
AI = AudioInference
using .AI: AudioSource, gammatonegram

In [ ]:
include("../worldmodel/smart_birth_death.jl")

In [ ]:
ground_truth, _ = generate(AI.generate_scene, AI.args, choicemap((:kernel => :n_tones, 4)))
initial_tr, weight = AI.generate_initial_tr(ground_truth, num_sources=0)
gram = AI.error_gram(initial_tr)
AI.plot_gtg(gram)

In [ ]:
tr, acc = mh(initial_tr, smart_birth_death_mh_kern; check=true)
println("acc: ", acc)

In [ ]:
AI.plot_gtg(underlying_gram(tr))

In [ ]:
total_acc = 0
for _=1:20
  tr, acc = mh(tr, smart_birth_death_mh_kern; check=true)
  total_acc += acc
end
println("Total number accepted: ", total_acc)

In [ ]:
function iterated_birth_death(tr, bd_kernel, accumulated_weight=0; check=true)
  (new_tr, weight) = bd_kernel(tr; check=check)
  accumulated_weight += weight
  stop_prob = exp(accumulated_weight) > 0.5 ? 0.99 : 0.25
  if bernoulli(stop_prob)
    return (new_tr, accumulated_weight)
  else
    return iterated_birth_death(new_tr, bd_kernel, accumulated_weight; check=check)
  end
end

In [ ]:
total_acc = 0
for _=1:20
  new_tr, weight = iterated_birth_death(tr, smart_birth_death_mh_kern; check=true)
  if bernoulli(weight)
    total_acc += 1
    tr = new_tr
  end
end
println("Total number accepted: ", total_acc)

In [ ]:
(new_tr, weight) = iterated_birth_death(tr, smart_birth_death_mh_kern; check=true)

In [ ]:
AI.plot_gtg(underlying_gram(tr))

In [ ]:
AI.plot_gtg(underlying_gram(new_tr))

In [ ]:
(proptr, _) = generate(smart_birth_death_proposal, (tr,))
get_choices(proptr)

In [ ]:
assess(smart_birth_death_proposal, (tr,), get_choices(proptr))

In [ ]:
newtr, weight, _, _ = update

In [ ]:
get_subtree(get_choices(tr), :world => :waves => AudioSource(2))

In [ ]:
(sum(underlying_gram(ground_truth)[:, 5]) - 1054)/64

In [ ]:
get_subtree(get_choices(ground_truth), :world)

In [ ]:
proptr, wt = generate(smart_birth_death_proposal, (initial_tr,), choicemap((:do_smart_birth, false)))
get_choices(proptr)

In [ ]:
first_pass_results = GenWorldModels.run_first_pass(smart_birth_death_involution, tr, proptr)
first_pass_results.update_spec.subspec

In [ ]:
newtr, wt, _, _ = update(tr, get_args(initial_tr),
map(_ -> NoChange(), get_args(initial_tr)),
first_pass_results.update_spec,
invert(first_pass_results.reverse_regenerated));
wt;

In [ ]:
wt

In [ ]:
get_subtree(get_choices(newtr), :world)

In [ ]:
wt

In [ ]:
AI.plot_gtg(underlying_gram(newtr))

In [ ]:
AI.plot_gtg(observed_gram(newtr))